In [1]:
import pandas as pd
import numpy as np
from math import ceil
import time
import glob,os
import re

In [2]:
DesignID=pd.read_csv('OptDesign.csv',sep=',',header=1)
DesignID=pd.get_dummies(data=DesignID,columns=['Full Braking Before Impact','Weather','Road Type','Day or Night'])
#Road=pd.get_dummies(DesignID['Road Type'])
#day=pd.get_dummies(DesignID['Day or Night'])
#weather=pd.get_dummies(DesignID['Weather'])
#day=pd.get_dummies(DesignID['Full Braking Before Impact'])
DesignID=DesignID.drop(columns=['Unnamed: 0'],axis=1)



In [6]:
def scantext():
    x=[]
    for path,dirs,files in os.walk('/Users/arash/Google Drive/AV-PED Inreraction/Data'): #for mac.os    #for ubuntu: os.walk('/home/arash/VRdata/Data/'):
        for f in files:
            if f.endswith('.txt'):
                x.append(os.path.join(path,f))
    return(x)
                

In [7]:
alltext=scantext()


In [8]:
experiments=[]
for filename in alltext:
    lastslashindex=filename.rfind('/')
    hyphen=filename[lastslashindex:].find('-')
    desID=int(filename[lastslashindex+1:][:hyphen-1])
    df=pd.read_csv(filename,header=None,names=['status','y','height','x',4,5,'o1','o2','o3','timeelaspsed','timehr','timemin','timesec'],sep=':|;|,|\(|\)',engine='python')   #x,y,height: coordinates,     #o1,o2,o3: head orientation
    df=df.drop(columns=[4,5],axis=1)
    

    df['scenario']=desID
    df['filename']=filename[24:]
    tindex=(df.index[df['status']=='u ']-1).tolist()
    pindex=(df.index[df['status']=='u ']).tolist()
    df.iloc[pindex,df.columns.get_loc('timeelaspsed')]=df.iloc[tindex,0].values
    df.iloc[pindex,df.columns.get_loc('timehr')]=df.iloc[tindex,1].values
    df.iloc[pindex,df.columns.get_loc('timemin')]=df.iloc[tindex,2].values
    df.iloc[pindex,df.columns.get_loc('timesec')]=df.iloc[tindex,3].values
    df=df.drop(df.index[tindex])
    df=df.reset_index()
    df=df.drop('index',axis=1)
    
    
    
    experiments.append(df)
    
#frame=pd.concat(list_,axis=0,ignore_index=True)    
                   


In [530]:
n=345
pindex=(experiments[n].index[experiments[n]['status']=='u ']).tolist()


for index, item in enumerate(pindex):
    if index>1:
        experiments[n].iloc[pindex[index-1]:pindex[index],7]=experiments[n].iloc[pindex[index-1],7]
experiments[n].iloc[max(pindex):,7]=experiments[n].iloc[max(pindex),7]        
    

#wait time calculation
firstcarL=experiments[n]['status'][experiments[n]['status']=='L0 '].index[0]
firstcarR=experiments[n]['status'][experiments[n]['status']=='R0 '].index[0]
firstcar=min(firstcarL,firstcarR)
firstcarappeartime=float(experiments[n].iloc[firstcar-1,7])
firstcarname=(experiments[n].iloc[firstcar,0])
firstcary0=experiments[n].iloc[firstcar,1]
if firstcary0>experiments[n].iloc[0,1]:
    firstcartime=float(experiments[n].iloc[experiments[n][experiments[n]['status']==firstcarname][experiments[n]['y']<experiments[n].iloc[0,1]].index[0]-1,7])
    firstcarindex=[experiments[n][experiments[n]['status']==firstcarname][experiments[n]['y']<experiments[n].iloc[0,1]].index[0]]
else:
    firstcartime=float(experiments[n].iloc[experiments[n][experiments[n]['status']==firstcarname][experiments[n]['y']>experiments[n].iloc[0,1]].index[0]-1,7])
    firstcarindex=[experiments[n][experiments[n]['status']==firstcarname][experiments[n]['y']>experiments[n].iloc[0,1]].index[0]]
    


pedafter1st=[i for i in pindex if i >= (firstcarindex[0])-1]
lastpedbefore1st=[i for i in pindex if i < (firstcarindex[0])-1]
#lastpedbefore1st=lastpedbefore1st[len(lastpedbefore1st)-1]

pedstartmovetime=float(experiments[n].iloc[pedafter1st][pd.to_numeric(experiments[n]['x'])<(float(experiments[n]['x'][0])-0.2)]['timeelaspsed'].tolist()[0])
pedstartmoveindex=experiments[n].iloc[pedafter1st][pd.to_numeric(experiments[n]['x'])<(float(experiments[n]['x'][0])-0.2)].index[0]

waittime=pedstartmovetime-firstcartime

#PET calculation
if 'ACCIDENT' in set(experiments[n]['status']): pet=0
else:
    pedpassLtime=float(experiments[n].iloc[pindex][pd.to_numeric(experiments[n]['x'])<(float(experiments[n]['x'][firstcarL+1]))]['timeelaspsed'].tolist()[0])
    CarLindex=experiments[n][experiments[n]['status'].str.contains('L',na=False)].index
    afterpedmoveCarLindex=[i for i in CarLindex if i > pedstartmoveindex]
    afterpedmoveCarLy=experiments[n].iloc[afterpedmoveCarLindex]['y']
    firstcarLy0=experiments[n].iloc[firstcarL,1]
    if firstcarLy0<experiments[n].iloc[0,1]:     #if L lane is directed right to left, ie one way scenarios
        possiblePetCarY=afterpedmoveCarLy[(afterpedmoveCarLy<experiments[n].iloc[0,1]) & (afterpedmoveCarLy>(experiments[n].iloc[0,1]-1))]      #cars one L lane that has passed a hypothetical one meter strip before pedestrian starting point, after pedestrian has crossed
        if possiblePetCarY.empty:
            pet=1000
        else:
            PetCarindex=possiblePetCarY.index[0]
            vehLpasstime=experiments[n].iloc[PetCarindex,7]
            pet=vehLpasstime-pedpassLtime
    else:    # two way scenarios
        possiblePetCarY=afterpedmoveCarLy[(afterpedmoveCarLy>experiments[n].iloc[0,1]) & (afterpedmoveCarLy<(experiments[n].iloc[0,1]+1))]
        if possiblePetCarY.empty:
            pet=1000
        else:
            PetCarindex=possiblePetCarY.index[0]
            vehLpasstime=experiments[n].iloc[PetCarindex,7]
            pet=vehLpasstime-pedpassLtime
    
print(pet)
print(waittime)

1000
0.0


/home/arash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/arash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/arash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/arash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:32: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/arash/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [537]:
experiments[n].iloc[170:222]
    
    
            

    
    


,status,y,height,x,o1,o2,o3,timeelaspsed,timehr,timemin,timesec,scenario,filename
170,NaN,282.2,0.1,647.6,NaN,NaN,NaN,9.326265,NaN,NaN,NaN,69,Carla/69--2018-10-7--14-58-08.txt
171,NaN,282.2,0.1,649.6,NaN,NaN,NaN,9.326265,NaN,NaN,NaN,69,Carla/69--2018-10-7--14-58-08.txt
172,NaN,287.0,0.1,647.6,NaN,NaN,NaN,9.326265,NaN,NaN,NaN,69,Carla/69--2018-10-7--14-58-08.txt
173,NaN,287.0,0.1,649.6,NaN,NaN,NaN,9.326265,NaN,NaN,NaN,69,Carla/69--2018-10-7--14-58-08.txt
174,u,290.1,2.0,653.4,0.9,0.0,-0.4,9.436471,2.0,58.0,18 PM,69,Carla/69--2018-10-7--14-58-08.txt
175,R0,286.1,0.1,648.6,NaN,NaN,NaN,9.436471,NaN,NaN,NaN,69,Carla/69--2018-10-7--14-58-08.txt
176,NaN,283.9,0.1,647.6,NaN,NaN,NaN,9.436471,NaN,NaN,NaN,69,Carla/69--2018-10-7--14-58-08.txt
177,NaN,283.9,0.1,649.6,NaN,NaN,NaN,9.436471,NaN,NaN,NaN,69,Carla/69--2018-10-7--14-58-08.txt
178,NaN,288.6,0.1,647.6,NaN,NaN,NaN,9.436471,NaN,NaN,NaN,69,Carla/69--2018-10-7--14-58-08.txt
179,NaN,288.6,0.1,649.6,NaN,NaN,NaN,9.436471,NaN,NaN,NaN,69,Carla/69--2018-10-7--14-58-08.txt


In [534]:
pedstartmovetime    
    
        

9.752707

In [535]:
firstcartime

9.752707

339

In [358]:
#DesignID['Road Type_One way'][experiments[56]['scenario'][0]]